In [3]:
import os
from gatenlp import Document
from gatenlp.corpora import ListCorpus
import requests
import json
import os
import ipywidgets as widgets
from IPython.display import display, Markdown
from gatenlp.lib_spacy import AnnSpacy
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import pandas as pd

from dotenv import load_dotenv


In [4]:
load_dotenv()

user_email = os.getenv("USEREMAIL")  # Enter your email here
password = os.getenv("PASSWORD")  # Enter your password here

# Fetch Access Token

# Define the URL for the authentication endpoint
auth_url = "http://localhost:8080/api/v1/auths/signin"

# Define the payload with user credentials
auth_payload = json.dumps({"email": user_email, "password": "admin"})

# Define the headers for the authentication request
auth_headers = {"accept": "application/json", "content-type": "application/json"}

# Make the POST request to fetch the access token
auth_response = requests.post(auth_url, data=auth_payload, headers=auth_headers)

# Extract the access token from the response
access_token = auth_response.json().get("token")

In [ ]:
def askChatbot(model, role, instruction, content):
    chat_url = "http://localhost:11434/api/chat"

    # Define the headers for the chat completion request, including the access token
    chat_headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}",
    }

    # Define the payload for the chat completion request
    chat_payload = json.dumps(
        {
            "stream": False,
            "model": model,
            "temperature": 0.0,
            "messages": [
                {"role": "system", "content": role},  # System role with additional context
                {"role": "user", "content": f"{instruction}\n\n{content}"},  # User message with instruction and text
            ],
        }
    )


    # Make the POST request to the chat completion endpoint
    chat_response = requests.post(chat_url, data=chat_payload, headers=chat_headers)
    #print(chat_response.json()["message"]["content"])
    return chat_response

In [ ]:
#chat_response = askChatbot("mistral:latest", "user", "What is Mendix?")
#print(chat_response.json()["message"]["content"])

 Mendix is a low-code platform for rapid application development. It allows both citizen developers and professional developers to build, deploy, and manage web and mobile applications without the need to write code from scratch. With Mendix, users can create, test, and deploy applications quickly using predefined templates, drag-and-drop interfaces, and reusable components. The platform is designed for building enterprise applications with a focus on speed, scalability, and flexibility.


In [6]:
# Create a new corpus with an empty list
corpus = ListCorpus([])

# Define the base directory
base_dir = "input/annotated"

# Walk through the directory and load each XML file
for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith(".xml"):
            file_path = os.path.join(root, file)
            doc = Document.load(file_path, fmt="gatexml")
            # Add the document to the corpus
            corpus.append(doc)
            print(f"Loaded {file_path} into corpus")            
                
print("All documents loaded into the corpus.")

Loaded input/annotated\dev\CASE OF ALTAY v. TURKEY (No. 2).xml into corpus
Loaded input/annotated\dev\CASE OF BELYAYEV AND OTHERS v. UKRAINE.xml into corpus
Loaded input/annotated\dev\CASE OF BIGUN v. UKRAINE.xml into corpus
Loaded input/annotated\test\CASE OF CABUCAK v. GERMANY.xml into corpus
Loaded input/annotated\test\CASE OF CAN v. TURKEY.xml into corpus
Loaded input/annotated\test\CASE OF CRISTIAN CATALIN UNGUREANU v. ROMANIA.xml into corpus
Loaded input/annotated\train\CASE OF DOKTOROV v. BULGARIA.xml into corpus
Loaded input/annotated\train\CASE OF EGILL EINARSSON v. ICELAND (No. 2).xml into corpus
Loaded input/annotated\train\CASE OF HOINESS v. NORWAY.xml into corpus
Loaded input/annotated\train\CASE OF KOSAITE - CYPIENE AND OTHERS v. LITHUANIA.xml into corpus
Loaded input/annotated\train\CASE OF LOZOVYYE v. RUSSIA.xml into corpus
Loaded input/annotated\train\CASE OF M.T. v. UKRAINE.xml into corpus
Loaded input/annotated\train\CASE OF MOSKALEV v. RUSSIA.xml into corpus
Loaded 

In [7]:
len(corpus)

30

In [8]:
print(doc.features.get("gate.SourceURL").replace("file:/C:/Users/mnavas/", "").replace("%20", " "))

CASE OF YEVGENIY ZAKHAROV v. RUSSIA.docx


In [ ]:
#nlp_spacy = English()
#nlp_spacy.add_pipe('sentencizer')
#tokenize = AnnSpacy(nlp_spacy, add_nounchunks=False, add_deps=False, add_entities=False)

#for doc in corpus:
#    doc = tokenize(doc)
#    doc

In [9]:
from gatenlp.visualization import CorpusViewer

viewer = CorpusViewer(corpus)
viewer.show()

In [ ]:
prompt = "Extract all dates from the text!"
doc_list = []

models = [
    "chevalblanc/claude-3-haiku:latest",
    "incept5/llama3.1-claude:latest",
    "llama3.3:latest",
    "deepseek-r1:8b",
    "mistral:latest",
]

for doc in corpus:
    doc_dict = {}
    print(f"Processing document: {doc.features.get("gate.SourceURL")}")    
    annotations = doc.annset("Section")
    procedure_annotations = annotations.with_type("Procedure")

    for ann in procedure_annotations:
        procedure_text = doc.text[ann.start:ann.end]
        print(f"Procedure text: {procedure_text}")

        for model in models:
            print(f"\nUsing model: {model}")
            chat_response = askChatbot(model, prompt, procedure_text)
            #print(chat_response.json()["message"]["content"])
            doc_dict["doc"] = doc.features.get("gate.SourceURL")
            doc_dict[model] = chat_response.json()["message"]["content"]
        
        doc_list.append(doc_dict)


Processing document: file:/C:/Users/mnavas/CASE%20OF%20ALTAY%20v.%20TURKEY%20(No.%202).docx
Procedure text: 1.  The case originated in an application (no. 11236/09) against the Republic of Turkey lodged with the Court under Article 34 of the Convention for the Protection of Human Rights and Fundamental Freedoms (“the Convention”) by a Turkish national, Mr Mehmet Aytunç Altay (“the applicant”), on 17 February 2006.
2.  The applicant was represented by Ms G. Tuncer, a lawyer practising in Istanbul. The Turkish Government (“the Government”) were represented by their Agent.
3.  The applicant alleged, in particular, that the restriction of the privacy of his consultations with his lawyer was incompatible with his rights under Article 8 of the Convention and that the domestic proceedings with respect to this measure had not complied with the requirements of Article 6 § 1 of the Convention.
4.  On 17 October 2017 notice of the above complaints was given to the Government and the remainder of 

In [ ]:
# Convert the results list to a pandas DataFrame
df = pd.DataFrame(doc_list)

# Display the DataFrame
print(df)

# Save the DataFrame to a CSV file for later analysis
df.to_csv("chat_responses.csv", index=False)

   doc                  chevalblanc/claude-3-haiku:latest  \
0       Here are all the dates extracted from the text...   
1       Extracted dates from the text:\n\n* 03/03/2010...   
2       Here are the dates extracted from the text:\n\...   
3       - 5 April 2016: The date the application was l...   
4       Dates extracted from the text:\n\n* 4 January ...   
5             - "25 November 2013"\n- "16 September 2014"   
6       **Dates Extracted:**\n\n* 18 February 2008 - "...   
7       Extracted dates from the text:\n\n* 16 June 20...   
8       Dates extracted from the text:\n\n* "3 June 20...   
9       Dates extracted from the text:\n\n* "19 Octobe...   
10      Extracted dates:\n\n* November 10, 2008 - "on ...   
11      Dates extracted from the text:\n\n* **20 Decem...   
12      Extracted dates:\n1. October 27, 2005 (the dat...   
13      Extracted dates:\n\n* 11 December 2015 - the d...   
14      Here are the dates extracted from the text:\n\...   
15      Dates extracted 